1. I'm a beginner and it is my first public notebook here
2. It is a very basic approach with RandomForest and SMOTE resampling. I will try to improve my work with some kinds of feature engineering later.
3. F1 score is about 42% and it is the best until now. Other notebooks with F1=99% are examples of incorrect validation techniques.
4. I will add some comments later

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score,recall_score
from sklearn.feature_selection import RFECV

In [ ]:
original = pd.read_csv('/kaggle/input/company-bankruptcy-prediction/data.csv')
original.head()

In [ ]:
y = original['Bankrupt?']

In [ ]:
original = original.drop(['Bankrupt?'], axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(original,y, test_size=0.33,stratify=y,random_state=42)

In [ ]:
os = SMOTE()

In [ ]:
columns = X_train.columns

In [ ]:
os_data_X,os_data_y=os.fit_resample(X_train, y_train)

In [ ]:
X_train = pd.DataFrame(data=os_data_X,columns=columns )

In [ ]:
y_train = pd.DataFrame(data=os_data_y,columns=['y'])

In [ ]:
y_train = os_data_y

In [ ]:
parameters = {'criterion':['entropy'], 'max_depth':range(11,17),'n_estimators':range(9,12),'class_weight':['balanced',None] }

In [ ]:
clf_rf = RandomForestClassifier(random_state=42)

In [ ]:
grid_search_cv_clf = GridSearchCV(clf_rf,parameters,cv=5, scoring='f1')

In [ ]:
grid_search_cv_clf.fit(X_train,y_train)

In [ ]:
grid_search_cv_clf.best_params_

In [ ]:
y_pred = grid_search_cv_clf.predict(X_test)

In [ ]:
recall = recall_score(y_test,y_pred)

In [ ]:
precision = precision_score(y_test,y_pred)

In [ ]:
F1 = 2 * (precision * recall) / (precision + recall)

In [ ]:
precision

In [ ]:
recall

In [ ]:
F1

RFE

In [ ]:
clf_rf = RandomForestClassifier(**grid_search_cv_clf.best_params_,random_state=42)

In [ ]:
cv_selector = RFECV(clf_rf,cv= 5, step=1,scoring='f1')

In [ ]:
cv_selector = cv_selector.fit(X_train, y_train)

In [ ]:
rfecv_mask = cv_selector.get_support()

In [ ]:
rfecv_features = [] 
for bool, feature in zip(rfecv_mask, X_train.columns):
 if bool:
     rfecv_features.append(feature)

In [ ]:
X_train_ref = X_train[rfecv_features]

In [ ]:
clf_rf.fit(X_train_ref,y_train)

In [ ]:
y_pred = clf_rf.predict(X_test[rfecv_features])

In [ ]:
recall = recall_score(y_test,y_pred)

In [ ]:
precision = precision_score(y_test,y_pred)

In [ ]:
F1 = 2 * (precision * recall) / (precision + recall)

In [ ]:
recall

In [ ]:
precision

In [ ]:
F1

RFE didn't improve the model quality  a lot :(